In [ ]:
import pandas as pd
import numpy as np

#url_data = (r"https://raw.githubusercontent.com/conversationai/unhealthy-conversations/main/corpus/train.csv")

#data_csv = pd.read_csv(url_data)

In [ ]:
#data_csv.head()

In [ ]:
url_data = (r"https://raw.githubusercontent.com/conversationai/unhealthy-conversations/main/unhealthy_full.csv")

data_csv = pd.read_csv(url_data)

In [ ]:
data_csv.head()

In [ ]:
data_csv.shape

In [ ]:
data_csv["comment"]

In [ ]:
data_csv_drop=data_csv.drop_duplicates(subset="comment")

In [ ]:
data_csv_drop.shape

In [ ]:
data_cnull= data_csv_drop.dropna()
data_cnull.shape

In [ ]:
data_del_ind= data_cnull[(data_cnull["sarcastic"]==0) & (data_cnull["healthy"]==0) & (data_cnull["antagonize"]==0) & (data_cnull["condescending"]==0) & (data_cnull["dismissive"]==0) & (data_cnull["hostile"]==0)].index
data_pnull = data_cnull.drop(data_del_ind)
data_pnull.shape

In [ ]:
data_wnull_ind=data_pnull[((data_cnull["antagonize"]==1) | (data_cnull["condescending"]==1) | (data_cnull["dismissive"]==1) | (data_cnull["hostile"]==1)) & (data_cnull["healthy"]==1)].index
data_wnull = data_pnull.drop(data_wnull_ind)
data_wnull.shape

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test= train_test_split(data_wnull, test_size=0.2, random_state=42)
print(data_train.shape, data_test.shape)

In [ ]:
import nltk

nltk.download('popular')

In [ ]:
import pandas as pd
import numpy as np
import string
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

embed_size = 300
max_features = 20000 
maxlen = 200 

print('Loading data...')

classes = ["sarcastic", "healthy", "antagonize", "condescending", "dismissive", "hostile"]
y = data_train[classes].values
y_test = data_test[classes].values

train_sentences = data_train["comment"]
test_sentences = data_test["comment"]

# print('Preprocessing train') 
# train = list()
# for i in train_sentences:
#     tokenizer = RegexpTokenizer(r'\w+')#removing punctuations
#     train.append([i.lower() for i in (tokenizer.tokenize(str(i))) if i not in stop_words])

# print('Preprocessing test')
# test = list()
# for i in test_sentences:
#     tokenizer = RegexpTokenizer(r'\w+')#removing punctuations
#     test.append([i.lower() for i in (tokenizer.tokenize(str(i))) if i not in stop_words])

train_sentences[:5]

In [ ]:
len(train_sentences)

In [ ]:
len(test_sentences)

In [ ]:
# 1. create custom embeddings 
words = list()
for i in train_sentences:
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(i) # word tokenizacia
    words.append(tokens) # pridá výsledné slová do prázdneho listu, ktorý sme na začiatku vytvorili
print('Word2Vec...')
print(words[:10])

In [ ]:
len(words[0])

In [ ]:
from gensim.models import Word2Vec
from keras.layers import Embedding

model = Word2Vec(words, min_count = 1, size = 300) #word to vector, implementacia z kniznice gensim
vocabulary = model.wv.vocab #slovnik
name = 'w2vPP.txt'
model.wv.save_word2vec_format(name, binary = False)

EMBEDDING_FILE = 'w2vPP.txt' 

In [ ]:
import tensorflow

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Input, Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, BatchNormalization, SpatialDropout1D, GlobalAveragePooling1D, concatenate, Activation, LSTM, Bidirectional
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_sentences))

tokenized_train_sentences = tokenizer.texts_to_sequences(train_sentences)
tokenized_test_sentences = tokenizer.texts_to_sequences(test_sentences)

train_padding = pad_sequences(tokenized_train_sentences, maxlen)
test_padding = pad_sequences(tokenized_test_sentences, maxlen)

def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))

print(list(embeddings_index.items())[:5])

In [ ]:
len(embeddings_index.get('society'))

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_matrix[:5]

In [ ]:
embedding_matrix.shape

In [ ]:
kfold = KFold(n_splits=3)
cvscores = []
accscores = []
rocscores = []

for train, test in kfold.split(train_padding, y):
    
    inputs = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inputs)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(6, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    print(model.summary())

    saved_model = "w2v.hdf5"
    checkpoint = ModelCheckpoint(saved_model, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

    print('Training model...')
    history = model.fit(train_padding, y, batch_size=32, epochs=4, callbacks=[checkpoint], validation_split=0.1)

    scores = model.evaluate(train_padding[test], y[test])
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

    y_pred = model.predict(test_padding)

    y_int = np.zeros_like(y_pred)
    y_int[y_pred > 0.5] = 1

    accuracy = accuracy_score(y_test,y_int)
    print('Accuracy is {}'.format(accuracy))
    accscores.append(accuracy)
    
    rocauc = roc_auc_score(y_test, y_pred)
    print('Roc-auc score is {}'.format(rocauc))
    rocscores.append(rocauc)
    
    print('Classification report {}'.format(classification_report(y_test, y_int, zero_division=0)))
        
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("Test accuracy is: {} %.2f (+/- %.2f)" %  (np.mean(accscores), np.std(accscores)))
print("Test roc-auc is: {} %.2f (+/- %.2f)" % (np.mean(rocscores), np.std(rocscores)))